In [1]:
import tensorflow as tf
import os
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
IMAGE_PIXELS = 784
learning_rate = 0.001
max_steps = 1000
fake_data = False
dropout = 0.9  # Keep prob for training dropout
log_dir = 'log'
data_dir = "MNIST_data/"

def deepnn(x):
    """deepnn builds the graph for a deep net for classifying digits.
    Args:
      x: an input tensor with the dimensions (N_examples, 784), where 784 is the
      number of pixels in a standard MNIST image.
    Returns:
      A tuple (y, keep_prob). y is a tensor of shape (N_examples, 10), with values
      equal to the logits of classifying the digit into one of 10 classes (the
      digits 0-9). keep_prob is a scalar placeholder for the probability of
      dropout.
    """
    # Reshape to use within a convolutional neural net.
    # Last dimension is for "features" - there is only one here, since images are
    # grayscale -- it would be 3 for an RGB image, 4 for RGBA, etc.
    with tf.name_scope('reshape'):
        x_image = tf.reshape(x, [-1, 28, 28, 1])

    # First convolutional layer - maps one grayscale image to 32 feature maps.
    with tf.name_scope('h_conv1'):
        with tf.name_scope('W'):
            W_conv1 = weight_variable([5, 5, 1, 32])
        with tf.name_scope('b'):
            b_conv1 = bias_variable([32])
        with tf.name_scope('relu_conv_Wx_plus_b'):
            h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

    # Pooling layer - downsamples by 2X.
    with tf.name_scope('h_pool1'):
        h_pool1 = max_pool_2x2(h_conv1)

    # Second convolutional layer -- maps 32 feature maps to 64.
    with tf.name_scope('h_conv2'):
        with tf.name_scope('W'):
            W_conv2 = weight_variable([5, 5, 32, 64])
        with tf.name_scope('b'):
            b_conv2 = bias_variable([64])
        with tf.name_scope('relu_conv_Wx_plus_b'):
            h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

    # Second pooling layer.
    with tf.name_scope('h_pool2'):
        h_pool2 = max_pool_2x2(h_conv2)

    # Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
    # is down to 7x7x64 feature maps -- maps this to 1024 features.
    with tf.name_scope('h_fc1'):
        with tf.name_scope('W'):
            W_fc1 = weight_variable([7 * 7 * 64, 1024])
        with tf.name_scope('b'):
            b_fc1 = bias_variable([1024])
        with tf.name_scope('h_pool2_flat'):
            h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
        with tf.name_scope('relu_Wx_plus_b'):
            h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

    # Dropout - controls the complexity of the model, prevents co-adaptation of
    # features.
    with tf.name_scope('h_fc1_drop'):
        with tf.name_scope('keep_prob'):
            keep_prob = tf.placeholder(tf.float32)
        with tf.name_scope('droppout'):
            h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
        tf.summary.scalar('dropout_keep_probability', keep_prob)

    # Map the 1024 features to 10 classes, one for each digit
    with tf.name_scope('h_fc2'):
        with tf.name_scope('W'):
            W_fc2 = weight_variable([1024, 10])
        with tf.name_scope('b'):
            b_fc2 = bias_variable([10])
        with tf.name_scope('Wx_plus_b'):
            y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    
    return y_conv, keep_prob


def conv2d(x, W):
    """conv2d returns a 2d convolution layer with full stride."""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    """max_pool_2x2 downsamples a feature map by 2X."""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')


def weight_variable(shape):
    """weight_variable generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    """bias_variable generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
    
def main():
    # Import data
    mnist = input_data.read_data_sets(data_dir, one_hot=True)

    # Create the model
    
    with tf.name_scope('input'):
        x = tf.placeholder(tf.float32, [None, 784])
        y_ = tf.placeholder(tf.float32, [None, 10])
        
    def get_feed_dict(mode, batch):
        d = {x: batch[0], y_: batch[1], keep_prob: 0.5}
        if mode == "train":
            d = {x: batch[0], y_: batch[1], keep_prob: 0.5}
        if mode == "test":
            d = {x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}
        if mode == "train accuracy":
            d = {x: batch[0], y_: batch[1], keep_prob: 1.0}

        #print("Returning feed dict: ", d)
        return d
        

    # Build the graph for the deep net
    y_conv, keep_prob = deepnn(x)

    # Define loss and optimizer
    with tf.name_scope('cross_entropy'):
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    tf.summary.scalar('cross_entropy', cross_entropy)
    
    with tf.name_scope('train_step'):
        train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)
    
    

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        # Merge all the summaries and write them out to logdir
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(os.path.join(log_dir,'train'), sess.graph)
        test_writer = tf.summary.FileWriter(os.path.join(log_dir,'test'))
        

        for i in range(20000):
            batch = mnist.train.next_batch(50)
            if i % 100 == 0:
                summary, train_accuracy = sess.run([merged, accuracy],feed_dict=get_feed_dict("train accuracy", batch))
                train_writer.add_summary(summary, i)
                print('step %d, training accuracy %g' % (i, train_accuracy))
                
                summary2, test_accuracy = sess.run([merged, accuracy],feed_dict=get_feed_dict("test", batch))
                test_writer.add_summary(summary2, i)
                print('step %d, test accuracy %g' % (i, test_accuracy))
            
            summary, _ = sess.run([merged, train_step],feed_dict=get_feed_dict("train", batch))
            train_writer.add_summary(summary, i)

        print('test accuracy %g' % accuracy.eval(feed_dict=get_feed_dict("test", batch)))


In [3]:
main()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy 0.18
step 0, test accuracy 0.1069
step 100, training accuracy 0.92
step 100, test accuracy 0.8471
step 200, training accuracy 0.96
step 200, test accuracy 0.9015
step 300, training accuracy 0.88
step 300, test accuracy 0.9328
step 400, training accuracy 0.9
step 400, test accuracy 0.9362
step 500, training accuracy 0.98
step 500, test accuracy 0.9459
step 600, training accuracy 0.94
step 600, test accuracy 0.9524
step 700, training accuracy 0.92
step 700, test accuracy 0.9537
step 800, training accuracy 0.96
step 800, test accuracy 0.958
step 900, training accuracy 0.96
step 900, test accuracy 0.9614
step 1000, training accuracy 0.98
step 1000, test accuracy 0.9638
step 1100, training accuracy 0.96
step 1100, test accuracy 0.9676
step 1200, training accuracy 0.92
step 1

KeyboardInterrupt: 